In [ ]:
# +-------------------------------------------------------------------------------------------------
# Author:cfolkers
# Ministry, Division, Branch: WLRS, GEOBC, Geospatial Services 
# Created Date: 2024/07/04
# Updated Date: 
# Description: quick script to summarize First Nations overlaps with the PG TSA and tsa overlap with FN
# +-------------------------------------------------------------------------------------------------

In [41]:
#load up 
import logging
import geopandas as gpd
import pandas as pd 
pd.options.display.float_format = '{:.6f}'.format
from bcgw2gdf import bcgw2gdf
import xlsxwriter
bcgw2gdf=bcgw2gdf()

In [42]:
#set up logging 
logging.basicConfig(level=logging.DEBUG)
debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

In [ ]:
#connect to BCGW from bcgw2gdf
bcgw2gdf.bcgw_connect()

In [118]:
#define queries for data 
pip_q ="""select * from WHSE_ADMIN_BOUNDARIES.PIP_CONSULTATION_AREAS_SP"""
tsa_q =""" select * from WHSE_ADMIN_BOUNDARIES.FADM_TSA where TSB_NUMBER IS NULL And RETIREMENT_DATE IS NULL And TSA_NUMBER_DESCRIPTION = 'Prince George TSA' """
WMFN_q ="""SELECT * FROM WHSE_ADMIN_BOUNDARIES.PIP_CONSULTATION_AREAS_SP where CNSLTN_AREA_LABEL IN (160,266) """
csfn_path=r'Spatial/CarrierSekani_TribalCouncil_Boundary.shp'

In [ ]:
#execute queries and load tables 
pip_in = bcgw2gdf.get_spatial_table(pip_q)
pg_tsa=bcgw2gdf.get_spatial_table(tsa_q)
WMFN=bcgw2gdf.get_spatial_table(WMFN_q)
CSFN=gpd.read_file('CSTC_FirstNations_Boundary.shp')

In [ ]:
#for offline use save files 

# pip_in.to_file('pip.gpkg', driver='GPKG', layer='pip')
# pg_tsa.to_file('pg_tsa.gpkg',driver='GPKG', layer='pg_tsa')
# WMFN.to_file('WMFN.gpkg',driver='GPKG', layer='WMFN')


In [ ]:
#if using local data read files using gpd 

# pip_in=gpd.read_file('pip.gpkg', layer='pip')
# pg_tsa=gpd.read_file('pg_tsa.gpkg', layer='pg_tsa')
# WMFN=gpd.read_file('WMFN.gpkg', layer='WMFN')
# CSFN=gpd.read_file('CSTC_FirstNations_Boundary.shp')

In [ ]:
#update CSFN cols and geom
CSFN['CNSLTN_AREA_NAME']='Carrier Sekani Tribal Council'
CSFN['OBJECTID_left']=123456789
CSFN.to_wkt()
CSFN.rename(columns={'Name':'CNSLTN_AREA_NAME', 'geometry':'wkt'}, inplace=True)
CSFN.set_geometry('wkt')

#merge WMFN and CSFN
joined=pd.concat([WMFN, CSFN], ignore_index=False)
joined.set_geometry('wkt', inplace=True)
joined=joined.dissolve()

#check to see data merged properly, 
ax=joined.plot(color='pink')
# CSFN.plot(ax=ax, color='yellow')
WMFN.plot(ax=ax, color='green')

In [193]:
#need to add CSFN to pip table 

#line below needed if using local data
# CSFN.rename(columns={'wkt':'geometry'}, inplace=True)

#use sjoin to return subset of pip_in with only boundaries that intersect with TSA
subset_pip= gpd.sjoin(pip_in, pg_tsa, how='inner', predicate='intersects')

subset_pip=pd.concat([subset_pip, CSFN], ignore_index=False)

#calculate new area and ha 
subset_pip['area']=subset_pip.area
subset_pip['Total Area of PIP Boundary (ha)']=subset_pip['area']/10000

#create list of columns to keep and use difference to drop those not in list 
pip_cols=['CNSLTN_AREA_NAME','Total Area of PIP Boundary (ha)','OBJECTID_left', 'wkt']
subset_pip.drop(subset_pip.columns.difference(pip_cols), axis= 1, inplace= True)

In [ ]:
#inspect data for only chosen columns 
# subset_pip[['CNSLTN_AREA_NAME','Total Area of PIP Boundary (ha)','OBJECTID_left']]
print(subset_pip.columns)

In [ ]:
#clip PIP boundaries to TSA
pip_clip=gpd.clip(subset_pip, pg_tsa,keep_geom_type=False)
print(pip_clip.columns)
#check to make sure features still match if
if len(pip_clip) == len(subset_pip):
    info(f"Features match: {len(subset_pip)}:{len(pip_clip)}")
    #calculate ha
    pip_clip['area']=pip_clip.area
    pip_clip['Area of PIP Boundary Overlapping PG TSA (ha)']=pip_clip['area']/10000
    # print(pip_clip.columns)
    #drop columns
    pip_cols=['Area of PIP Boundary Overlapping PG TSA (ha)','OBJECTID_left', 'wkt']
    pip_clip.drop(pip_clip.columns.difference(pip_cols), axis= 1, inplace= True)
    # print(pip_clip.columns)
else:
     warning('features do not match! check your work!')

In [ ]:
print(subset_pip.columns)
print(pip_clip.columns)
pip_clip

In [ ]:
#from the sjoin above it added both objectid columns for the left and right gdf, rename left objectid
# subset_pip.rename(columns={'OBJECTID_left':'OBJECTID'}, inplace=True)

#join gdfs on objectid returned from bcgw
pip_out=pd.merge(subset_pip,pip_clip, on='OBJECTID_left', how='outer')
pip_out.drop(columns='wkt_x', inplace=True)

#check number of rows
debug(f"{len(pip_out)} features after join")

In [ ]:
pip_out

In [ ]:
pip_out.rename(columns={'OBJECTID_left':'OBJECTID','wkt_y':'wkt' }, inplace=True)
pip_out=pip_out.set_geometry('wkt')
pip_out.columns

In [199]:
# redundant but good to check work with new tools

tsa_clipped=[]
#loop through PIP boundaries and clip the TSA to any that apply
for idx, boundary in pip_in.iterrows():
    clipped_gdf = gpd.clip(pg_tsa, boundary.wkt)
    #add new attributes from PIP layers to TSA layer
    # clipped_gdf['CNSLTN_AREA_NAME'] = boundary['CNSLTN_AREA_NAME']
    clipped_gdf['OBJECTID'] = boundary['OBJECTID']
    #append clipped gdf to list
    tsa_clipped.append(clipped_gdf)
    
#combine list of gdfs into one     
result_gdf = gpd.GeoDataFrame(pd.concat(tsa_clipped, ignore_index=True))

#drop cols that do not exist in list 
result_cols=['Area of TSA in PIP Boundary','CNSLTN_AREA_NAME','wkt','OBJECTID']
result_gdf.drop(result_gdf.columns.difference(result_cols), axis= 1, inplace= True)

#calculate ha
result_gdf['area']=result_gdf.area
result_gdf['Area of TSA in PIP Boundary(ha)']=result_gdf['area']/10000

#check number of rows
debug(f"{len(result_gdf)} features after clipping")

DEBUG:root:57 features after clipping


In [ ]:
print(pip_out.columns)
print(result_gdf.columns)

In [ ]:
# inspect data for one record
filtered=result_gdf[result_gdf['OBJECTID']==784332]
ax=pg_tsa.plot( color='orange')
filtered.plot(ax=ax, edgecolor='green', facecolor='none', linewidth=2.0)

In [202]:
#final merge on objectid 
pip_out= pip_out.merge(result_gdf, on='OBJECTID', how='outer')
pip_out.rename(columns={'wkt_x':'wkt', 'Area of TSA in PIP Boundary(ha)_x':'Area of TSA in PIP Boundary(ha)'}, inplace=True)


In [ ]:
pip_out

In [ ]:
pip_out.columns

In [207]:
#drop cols 
pip_cols=['CNSLTN_AREA_NAME', 'OBJECTID', 'Total Area of PIP Boundary (ha)',
       'wkt', 'Area of PIP Boundary Overlapping PG TSA (ha)','Area of TSA in PIP Boundary(ha)']

pip_out.drop(pip_out.columns.difference(pip_cols), axis= 1, inplace= True)

In [ ]:
pip_out.rename(columns={'OBJECTID_left':'OBJECTID'}, inplace=True)
#calculate ha for tsa 
pg_tsa['area']=pg_tsa.area
pg_tsa['ha']=pg_tsa['area']/10000

#assign ha to var
pg_ha=pg_tsa['ha'].values[0]
debug(pg_ha)

#calculate percents
pip_out['% Overlap with PG TSA']=pip_out['Area of PIP Boundary Overlapping PG TSA (ha)']/pg_ha*100
pip_out['% of PG TSA overlap over FN']=pip_out['Area of TSA in PIP Boundary(ha)']/ pip_out['Total Area of PIP Boundary (ha)']*100

#drop columns not needed for spreadsheet
# drop_cols=['wkt', 'area',]
# pip_out.drop(columns=drop_cols, axis=1, inplace=True)


print(pip_out.columns)
#reorganize columns 
pip_out=pip_out.reindex(columns=['OBJECTID', 'CNSLTN_AREA_NAME',
                                'Total Area of PIP Boundary (ha)','Area of PIP Boundary Overlapping PG TSA (ha)',
                                'Area of TSA in PIP Boundary(ha)','% Overlap with PG TSA','% of PG TSA overlap over FN'])

In [210]:
#export df with text at the top 

outpath = r'/deliverables/PG_TSA_FN_Territory_Overlaps.xlsx'
header_txt='CONFIDENTIAL, FOR INTERNAL USE ONLY'
with pd.ExcelWriter(outpath, engine='xlsxwriter') as writer:
    pip_out.to_excel(writer, sheet_name='Overlaps', header=True, index=False, startrow=2)
    
    workbook = writer.book
    worksheet = writer.sheets['Overlaps']
    
    header_format = workbook.add_format({
        'bold': True,
        'font_color': 'red',
        'font_size': 24
    })
    
    worksheet.write('A1', header_txt, header_format)


# Area with CSFN and WMFN removed 

In [ ]:
subset_pip.columns
# subset_pip.plot()
print(len(subset_pip))

In [218]:
# #use sjoin to return subset of pip_in with only boundaries that intersect with TSA
# subset_pip= gpd.sjoin(pip_in, pg_tsa, how='inner', predicate='intersects')
# subset_pip.rename(columns={'OBJECTID_left':'OBJECTID'}, inplace=True)
#remove WMFN and CSFN from all pip layers and calculate new areas 
pip_remove=gpd.overlay(subset_pip,joined, how='difference')

pip_remove['area']=pip_remove.area
pip_remove['Total Area of PIP Boundary with WMFN and CSFN Removed']=pip_remove['area']/10000



#remove WMFN from pg_tas
tsa_remove=gpd.overlay(pg_tsa,joined, how='difference')

In [ ]:
ax=pg_tsa.plot()
tsa_remove.plot(ax=ax,color='red')
# joined.plot(ax=ax, color='yellow')
pip_remove.plot(color='pink')

In [ ]:
print(pip_remove.columns)

In [223]:

#sjoin to new tsa shape 
subset_pip= gpd.sjoin(pip_remove, tsa_remove, how='inner', predicate='intersects')
subset_pip.rename(columns={'OBJECTID_left':'OBJECTID'}, inplace=True)

#clip new pip to new tsa shape 
pip_remove_clip=gpd.clip(subset_pip, tsa_remove,keep_geom_type=False)

pip_remove_clip['area']=pip_remove_clip.area
pip_remove_clip['Area of PIP Boundary Overlapping PG TSA (ha) (with WMFN and CSFN Removed)']=pip_remove_clip['area']/10000


In [ ]:
print(len(pip_remove_clip))
print(pip_remove_clip.columns)

In [233]:
#drop columns 
pip_cols=['OBJECTID','CNSLTN_AREA_NAME','Total Area of PIP Boundary (ha)',
            'Total Area of PIP Boundary with WMFN and CSFN Removed',
            'Area of PIP Boundary Overlapping PG TSA (ha) (with WMFN and CSFN Removed)']
pip_remove_clip.drop(pip_remove_clip.columns.difference(pip_cols), axis= 1, inplace= True)

In [ ]:
pip_remove_clip.columns

In [ ]:

tsa_remove['area']=tsa_remove.area
tsa_remove['ha']=tsa_remove['area']/10000

#assign ha to var
tsa_remove_ha=tsa_remove['ha'].values[0]
debug(tsa_remove)

#calculate percents
pip_remove_clip['% Overlap with PG TSA(without WMFN & CSFN)']=pip_remove_clip['Area of PIP Boundary Overlapping PG TSA (ha) (with WMFN and CSFN Removed)']/tsa_remove_ha*100

pip_remove_clip['% of PG TSA overlap over FN']=pip_remove_clip['Area of PIP Boundary Overlapping PG TSA (ha) (with WMFN and CSFN Removed)']/ pip_remove_clip['Total Area of PIP Boundary (ha)']*100

#drop columns not needed for spreadsheet
# drop_cols=['wkt', 'area',]
pip_remove_clip.drop(columns='OBJECTID', inplace=True)
pip_remove_clip=pip_remove_clip.reindex(columns=['OBJECTID','CNSLTN_AREA_NAME', 'Total Area of PIP Boundary (ha)',
       'Total Area of PIP Boundary with WMFN and CSFN Removed',
       'Area of PIP Boundary Overlapping PG TSA (ha) (with WMFN and CSFN Removed)',
       '% Overlap with PG TSA(without WMFN & CSFN)',
       '% of PG TSA overlap over FN'])

In [ ]:
pip_remove_clip.columns

In [243]:
outpath = r'/deliverables/PG_TSA_FN_Territory_Overlaps_Without_WMFN_CSFN.xlsx'
header_txt='CONFIDENTIAL, FOR INTERNAL USE ONLY'
with pd.ExcelWriter(outpath, engine='xlsxwriter') as writer:
    pip_remove_clip.to_excel(writer, sheet_name='Overlaps', header=True, index=False, startrow=2)
    
    workbook = writer.book
    worksheet = writer.sheets['Overlaps']
    
    header_format = workbook.add_format({
        'bold': True,
        'font_color': 'red',
        'font_size': 24
    })
    
    worksheet.write('A1', header_txt, header_format)
